**Библиотеки**

In [1]:
!apt-get -y update
!apt-get -y install gfortran gcc g++ make git meson ninja-build libblas-dev liblapack-dev

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,648 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [86.7 kB]
Get:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:14 http://a

In [2]:
!pip install pycutest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.3 MB/s eta 0:00:00


In [3]:
!pip -q install -U "meson>=0.62" ninja
!meson --version
!which meson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 6.1 MB/s eta 0:00:00
1.10.1
/usr/local/bin/meson


In [ ]:
%%bash
set -e

PREFIX=/content/opt/cutest
SRC=/content/src/cutest_build
mkdir -p "$SRC" "$PREFIX"
cd "$SRC"

git clone --depth 1 https://github.com/ralna/SIFDecode sifdecode
cd sifdecode
meson setup builddir --prefix="$PREFIX"
meson compile -C builddir
meson install -C builddir
cd ..

git clone --depth 1 https://github.com/ralna/CUTEst cutest
cd cutest
meson setup builddir --prefix="$PREFIX"
meson compile -C builddir
meson install -C builddir

The Meson build system
Version: 1.10.1
Source dir: /content/src/cutest_build/sifdecode
Build dir: /content/src/cutest_build/sifdecode/builddir
Build type: native build
Project name: SIFDecode
Project version: 3.1.0
Fortran compiler for the host machine: gfortran (gcc 11.4.0 "GNU Fortran (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0")
Fortran linker for the host machine: gfortran ld.bfd 2.38
Host machine cpu family: x86_64
Host machine cpu: x86_64
Build targets in project: 3

SIFDecode 3.1.0

  User defined options
    prefix: /content/opt/cutest

Found ninja-1.13.0.git.kitware.jobserver-pipe-1 at /usr/local/bin/ninja
ninja: Entering directory `/content/src/cutest_build/sifdecode/builddir'
[1/14] Scanning target clsf for modules
[2/14] Scanning target slct for modules
[3/14] Generating dynamic dependency information for target clsf
[4/14] Scanning target sifdecoder for modules
[5/14] Generating dynamic dependency information for target slct
[6/14] Generating dynamic dependency information for

Cloning into 'sifdecode'...
Cloning into 'cutest'...


In [ ]:
import os

PREFIX = "/content/opt/cutest"
os.environ["SIFDECODE"] = PREFIX
os.environ["CUTEST"]    = PREFIX

os.environ["MASTSIF"]   = "/content/mastsif_local"
os.makedirs(os.environ["MASTSIF"], exist_ok=True)

os.environ["PATH"] = f"{PREFIX}/bin:" + os.environ.get("PATH","")
os.environ["LD_LIBRARY_PATH"] = f"{PREFIX}/lib:" + os.environ.get("LD_LIBRARY_PATH","")

print("CUTEST =", os.environ["CUTEST"])
print("SIFDECODE =", os.environ["SIFDECODE"])
print("MASTSIF =", os.environ["MASTSIF"])

CUTEST = /content/opt/cutest
SIFDECODE = /content/opt/cutest
MASTSIF = /content/mastsif_local


In [6]:
!pip -q install pycutest

In [7]:
import pycutest
print("pycutest ok:", pycutest.__version__)

pycutest ok: 1.8.0


/usr/local/lib/python3.12/dist-packages/pycutest/__init__.py:26: RuntimeWarning: the PYCUTEST_CACHE environment variable is not set; current folder will be used for caching.
  warnings.warn("the PYCUTEST_CACHE environment variable is not set; current folder will be used for caching.", RuntimeWarning)


In [8]:
import logging
import sys
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Callable, Tuple, List, Optional

import numpy as np
import sympy as sp
import jax.numpy as jnp
from jax import jacfwd
from sympy import Eq, Rel

In [9]:
@dataclass
class OptimizationConfig:
    max_newton_iterations: int = 1000

    mu_factor: float = 0.1
    mu_min: float = 1e-6
    residual_tolerance_factor: float = 0.1

    s_margin: float = 1e-6
    max_step_reductions: int = 30
    step_reduction_factor: float = 0.5
    min_step_size: float = 1e-10

    input_file_path: Optional[str] = None
    log_to_file: bool = True


In [10]:
@dataclass
class OptimizationProblem:
    target_function: Callable
    target_expression: sp.Expr
    x_symbols: List[sp.Symbol]
    equalities: List[sp.Expr]
    inequalities: List[sp.Expr]
    lambda_symbols: List[sp.Symbol]
    nu_symbols: List[sp.Symbol]
    s_symbols: List[sp.Symbol]

In [ ]:
@dataclass
class InitialPoint:
    x: np.ndarray
    s: np.ndarray
    lambdas: np.ndarray
    nus: np.ndarray
    mu: float
    epsilon: float


In [12]:
def setup_logger(log_to_file: bool) -> logging.Logger:
    logger = logging.getLogger("interior_point_optimizer")
    logger.setLevel(logging.INFO)
    logger.handlers.clear()
    logger.propagate = False

    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(logging.Formatter("%(message)s"))
    logger.addHandler(console_handler)

    if log_to_file:
        logs_dir = Path("logs")
        logs_dir.mkdir(exist_ok=True)

        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_path = logs_dir / f"optimization_log_{timestamp}.txt"

        file_handler = logging.FileHandler(file_path, encoding="utf-8")
        file_handler.setFormatter(logging.Formatter("%(message)s"))
        logger.addHandler(file_handler)

    return logger

In [13]:
def parse_constraint(constraint_string: str) -> Tuple[Optional[str], Optional[sp.Expr]]:
    constraint_string = constraint_string.strip()

    try:
        if "=" in constraint_string and all(op not in constraint_string for op in [">=", "<=", "!="]):
            constraint_string = constraint_string.replace("==", "=")
            left, right = constraint_string.split("=", 1)
            lhs = sp.sympify(left)
            rhs = sp.sympify(right)
            return "equality", lhs - rhs

        expr = sp.sympify(constraint_string)

        while isinstance(expr, sp.Not) and isinstance(expr.args[0], sp.Eq):
            expr = expr.args[0]

        if isinstance(expr, Eq):
            return "equality", expr.lhs - expr.rhs
        elif isinstance(expr, Rel):
            return "inequality", expr.lhs - expr.rhs

        return None, None

    except Exception as e:
        logging.error(f"Error parsing constraint '{constraint_string}': {e}")
        return None, None

In [ ]:
def read_problem_from_file(file_path: Path) -> Tuple[OptimizationProblem, InitialPoint]:
    if not file_path.exists():
        raise FileNotFoundError(f"File '{file_path}' does not exist!")

    x, s, lambdas, nus, mu, epsilon = None, None, None, None, None, None
    equalities, inequalities = [], []
    target_expr = None
    in_constraints_section = False

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()

            if not line or line.startswith("#"):
                continue

            if line.lower().startswith("constraints"):
                in_constraints_section = True
                continue

            if in_constraints_section:
                constraint_type, expr = parse_constraint(line)
                if constraint_type == "equality":
                    equalities.append(expr)
                elif constraint_type == "inequality":
                    inequalities.append(expr)
                continue

            if ":" not in line:
                continue

            key, value = line.split(":", 1)
            key = key.strip()
            value = value.strip()

            if key == "target_function":
                target_expr = sp.sympify(value)
            elif key == "x":
                x = np.array([float(v) for v in value.split()], dtype=float)
            elif key == "s":
                s = np.array([float(v) for v in value.split()], dtype=float)
            elif key == "lambdas":
                lambdas = np.array([float(v) for v in value.split()], dtype=float)
            elif key == "nus":
                nus = np.array([float(v) for v in value.split()], dtype=float)
            elif key == "mu":
                mu = float(value)
            elif key in ("epsilon", "eps"):
                epsilon = float(value)

    if target_expr is None:
        raise ValueError("target_function is missing in the input file")
    if x is None or s is None or lambdas is None or mu is None or epsilon is None:
        raise ValueError("Input file must contain x, s, lambdas, mu, and epsilon (nus is optional)")

    all_syms = set(target_expr.free_symbols)
    for expr in equalities + inequalities:
        all_syms |= set(expr.free_symbols)
    x_symbols = sorted(all_syms, key=lambda sym: sym.name)

    lambda_symbols = sp.symbols(f'λ1:{len(equalities) + 1}')
    nu_symbols = sp.symbols(f'ν1:{len(inequalities) + 1}')
    s_symbols = sp.symbols(f's1:{len(inequalities) + 1}')

    if len(s) != len(inequalities):
        raise ValueError(f"Length of s ({len(s)}) must equal number of inequalities ({len(inequalities)})")
    if len(lambdas) != len(equalities):
        raise ValueError(f"Length of lambdas ({len(lambdas)}) must equal number of equalities ({len(equalities)})")

    if np.any(s <= 0.0):
        bad = np.where(s <= 0.0)[0] + 1
        raise ValueError(f"Initial slack variables must satisfy s_j > 0. Non-positive indices: {bad.tolist()}")

    if nus is None:
        nus = np.array(mu / s, dtype=float)

    if len(nus) != len(inequalities):
        raise ValueError(f"Length of nus ({len(nus)}) must equal number of inequalities ({len(inequalities)})")

    if len(inequalities) > 0:
        g_funcs = [sp.lambdify(x_symbols, g, modules="numpy") for g in inequalities]
        gx = np.array([float(gf(*x)) for gf in g_funcs], dtype=float)
        violation = gx + s

        if np.linalg.norm(violation, ord=np.inf) > 1e-4:
            logging.warning(
                "Initial point is poorly consistent with g(x)+s=0. "
                f"Max |g(x)+s| = {float(np.max(np.abs(violation))):.3e}. "
                "Consider adjusting initial s or x."
            )

    problem = OptimizationProblem(
        target_function=sp.lambdify(x_symbols, target_expr, modules="numpy"),
        target_expression=target_expr,
        x_symbols=x_symbols,
        equalities=equalities,
        inequalities=inequalities,
        lambda_symbols=list(lambda_symbols),
        nu_symbols=list(nu_symbols),
        s_symbols=list(s_symbols)
    )

    initial = InitialPoint(x=x, s=s, lambdas=lambdas, nus=nus, mu=mu, epsilon=epsilon)
    return problem, initial


In [15]:
def create_lagrangian(
    problem: OptimizationProblem,
    mu: float
) -> sp.Expr:
    L = problem.target_expression

    L -= mu * sum(sp.log(s) for s in problem.s_symbols)

    for lam, h in zip(problem.lambda_symbols, problem.equalities):
        L += lam * h

    for nu, g, s in zip(problem.nu_symbols, problem.inequalities, problem.s_symbols):
        L += nu * (g + s)

    nu_substitution = {nu: mu / s for nu, s in zip(problem.nu_symbols, problem.s_symbols)}
    L = L.subs(nu_substitution)

    return L

In [ ]:
def create_kkt_system(
    lagrangian: sp.Expr,
    x_symbols: List[sp.Symbol],
    s_symbols: List[sp.Symbol],
    lambda_symbols: List[sp.Symbol],
    nu_symbols: List[sp.Symbol],
) -> Callable:
    grad_x = [sp.diff(lagrangian, x) for x in x_symbols]
    grad_lambda = [sp.diff(lagrangian, lam) for lam in lambda_symbols]
    grad_nu = [sp.diff(lagrangian, nu) for nu in nu_symbols]
    grad_s = [sp.diff(lagrangian, s) for s in s_symbols]

    system_equations = grad_x + grad_lambda + grad_nu + grad_s
    all_symbols = list(x_symbols) + list(lambda_symbols) + list(nu_symbols) + list(s_symbols)

    system_func = sp.lambdify(all_symbols, system_equations, modules="jax")

    def kkt_residual(y: jnp.ndarray) -> jnp.ndarray:
        return jnp.array(system_func(*y))

    return kkt_residual

In [ ]:
def solve_interior_point(
    problem: OptimizationProblem,
    initial: InitialPoint,
    config: OptimizationConfig,
    logger: logging.Logger
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, float]:
    x_dim = len(initial.x)
    lambda_dim = len(initial.lambdas)
    nu_dim = len(initial.nus)
    s_dim = len(initial.s)

    x = np.array(initial.x, dtype=float)
    lambdas = np.array(initial.lambdas, dtype=float)
    nus = np.array(initial.nus, dtype=float)
    s = np.array(initial.s, dtype=float)

    if np.any(s <= config.s_margin):
        bad = np.where(s <= config.s_margin)[0] + 1
        raise ValueError(
            f"Initial slack variables must satisfy s_j > u, u={config.s_margin}. Bad indices: {bad.tolist()}"
        )

    mu = float(initial.mu)

    outer_iter = 0
    while mu > config.mu_min:
        outer_iter += 1
        logger.info("=" * 80)
        logger.info(f"Outer iteration {outer_iter}: solving barrier problem with μ = {mu:.3e}")
        logger.info("=" * 80)

        lagrangian = create_lagrangian(problem, mu)
        kkt_system = create_kkt_system(
            lagrangian,
            problem.x_symbols,
            problem.s_symbols,
            problem.lambda_symbols,
            problem.nu_symbols,
        )

        y = jnp.array(np.concatenate([x, lambdas, nus, s]))

        for iteration in range(config.max_newton_iterations):
            F = kkt_system(y)
            residual = float(jnp.linalg.norm(F))

            logger.info(f"Residual: {residual:.6e}")

            if residual < config.residual_tolerance_factor * mu:
                break

            J = jacfwd(kkt_system)(y)
            delta = jnp.linalg.solve(J, -F)

            step = 1.0
            for attempt in range(config.max_step_reductions):
                y_candidate = y + step * delta
                s_new = np.array(y_candidate[-s_dim:], dtype=float)

                if np.all(s_new > config.s_margin):
                    break

                step *= config.step_reduction_factor
                logger.info(
                    f"Slack would violate s>u (u={config.s_margin}). "
                    f"Reducing step: attempt {attempt + 1}, step={step:.3e}"
                )

                if step < config.min_step_size:
                    logger.warning(
                        f"Step became too small (step={step:.3e} < {config.min_step_size}). "
                        "Stopping inner iterations."
                    )
                    break
            else:
                logger.warning(
                    "Could not find a step maintaining s>u within the max number of reductions. "
                    "Stopping inner iterations."
                )
                break

            if step < config.min_step_size:
                break

            y = y_candidate

        else:
            logger.info("Maximum Newton iterations reached for current μ.")

        y_np = np.array(y, dtype=float)
        x = y_np[:x_dim]
        lambdas = y_np[x_dim:x_dim + lambda_dim]
        nus = y_np[x_dim + lambda_dim:x_dim + lambda_dim + nu_dim]
        s = y_np[x_dim + lambda_dim + nu_dim:]

        if np.any(s <= config.s_margin):
            bad = np.where(s <= config.s_margin)[0] + 1
            raise RuntimeError(
                f"After inner solve, slack left feasible region s>u. "
                f"Bad indices: {bad.tolist()}, min(s)={float(np.min(s)):.3e}"
            )

        mu *= config.mu_factor

    logger.info("Barrier loop finished: μ <= μ_min")
    return x, lambdas, nus, s, mu


In [18]:
def log_initial_parameters(
    initial: InitialPoint,
    problem: OptimizationProblem,
    logger: logging.Logger
) -> None:
    logger.info('Initial Parameters:\n')

    logger.info('Initial approximation x:')
    for i, val in enumerate(initial.x, 1):
        logger.info(f'x{i} = {val}')

    logger.info('\nInitial slack variables s:')
    for i, val in enumerate(initial.s, 1):
        logger.info(f's{i} = {val}')

    logger.info('\nInitial equality multipliers λ:')
    for i, val in enumerate(initial.lambdas, 1):
        logger.info(f'λ{i} = {val}')

    if len(initial.nus) > 0:
        logger.info('\nInitial inequality multipliers ν:')
        for i, val in enumerate(initial.nus, 1):
            logger.info(f'ν{i} = {val}')

    logger.info(f'\nInitial μ = {initial.mu}')
    logger.info(f'Initial ε = {initial.epsilon}\n')


In [19]:
def log_solution(
    x_opt: np.ndarray,
    lambda_opt: np.ndarray,
    nu_opt: np.ndarray,
    s_opt: np.ndarray,
    mu_final: float,
    target_function: Callable,
    logger: logging.Logger
) -> None:
    logger.info('Optimal Solution:')

    for i, val in enumerate(x_opt, 1):
        logger.info(f'x{i}* = {val}')

    if len(lambda_opt) > 0:
        logger.info('\nOptimal equality multipliers λ:')
        for i, val in enumerate(lambda_opt, 1):
            logger.info(f'λ{i}* = {val}')

    if len(nu_opt) > 0:
        logger.info('\nOptimal inequality multipliers ν:')
        for i, val in enumerate(nu_opt, 1):
            logger.info(f'ν{i}* = {val}')

    if len(s_opt) > 0:
        logger.info('\nOptimal slack variables s:')
        for i, val in enumerate(s_opt, 1):
            logger.info(f's{i}* = {val}')

    logger.info(f'\nFinal μ = {mu_final:.3e}')

    try:
        fval = float(target_function(*x_opt))
        logger.info(f'Objective value at optimum: {fval}')
    except Exception as e:
        logger.info(f'Objective value at optimum: <could not evaluate: {e}>')


In [ ]:
from dataclasses import dataclass
from typing import Dict, Any

@dataclass
class CutestProblemWrapper:
    prob: Any
    meq: int
    mineq: int

    trans_is_var_bound: np.ndarray
    trans_orig_index: np.ndarray
    trans_sign: np.ndarray
    trans_var_index: np.ndarray
    trans_var_is_upper: np.ndarray

    eq_orig_index: np.ndarray

def _is_effectively_infinite(v: float, tol: float = 1e19) -> bool:
    return abs(float(v)) >= tol

def _build_cutest_wrapper(prob: Any) -> Tuple[CutestProblemWrapper, InitialPoint]:
    n = int(prob.n)
    m = int(getattr(prob, "m", 0))

    cl = np.array(getattr(prob, "cl", np.zeros(m)), dtype=float) if m > 0 else np.zeros(0)
    cu = np.array(getattr(prob, "cu", np.zeros(m)), dtype=float) if m > 0 else np.zeros(0)

    eq_orig = []
    ineq_entries = []

    if m > 0:
        for i in range(m):
            cli, cui = float(cl[i]), float(cu[i])
            if (not _is_effectively_infinite(cli)) and (not _is_effectively_infinite(cui)) and abs(cli - cui) <= 0.0:
                eq_orig.append(i)
            else:
                if not _is_effectively_infinite(cui):
                    ineq_entries.append({"type": "orig", "i": i, "sign": +1.0, "bound": cui})
                if not _is_effectively_infinite(cli):
                    ineq_entries.append({"type": "orig", "i": i, "sign": -1.0, "bound": cli})

    bl = np.array(getattr(prob, "bl", np.full(n, -1e20)), dtype=float)
    bu = np.array(getattr(prob, "bu", np.full(n, +1e20)), dtype=float)

    for j in range(n):
        if not _is_effectively_infinite(float(bu[j])):
            ineq_entries.append({"type": "var", "j": j, "is_upper": True, "bound": float(bu[j])})
        if not _is_effectively_infinite(float(bl[j])):
            ineq_entries.append({"type": "var", "j": j, "is_upper": False, "bound": float(bl[j])})

    meq = len(eq_orig)
    mineq = len(ineq_entries)

    trans_is_var = np.zeros(mineq, dtype=bool)
    trans_orig_index = np.zeros(mineq, dtype=int)
    trans_sign = np.ones(mineq, dtype=float)
    trans_var_index = np.zeros(mineq, dtype=int)
    trans_var_is_upper = np.zeros(mineq, dtype=bool)

    for k, ent in enumerate(ineq_entries):
        if ent["type"] == "orig":
            trans_is_var[k] = False
            trans_orig_index[k] = int(ent["i"])
            trans_sign[k] = float(ent["sign"])
        else:
            trans_is_var[k] = True
            trans_var_index[k] = int(ent["j"])
            trans_var_is_upper[k] = bool(ent["is_upper"])

    wrapper = CutestProblemWrapper(
        prob=prob,
        meq=meq,
        mineq=mineq,
        trans_is_var_bound=trans_is_var,
        trans_orig_index=trans_orig_index,
        trans_sign=trans_sign,
        trans_var_index=trans_var_index,
        trans_var_is_upper=trans_var_is_upper,
        eq_orig_index=np.array(eq_orig, dtype=int),
    )

    x0 = np.array(prob.x0, dtype=float).copy()

    g0 = _cutest_eval_g(wrapper, x0)
    s0 = np.maximum(-g0 + 1.0, 1e-3)
    mu0 = 1e-1
    nu0 = mu0 / s0
    lam0 = np.zeros(meq, dtype=float)

    initial = InitialPoint(x=x0, s=s0, lambdas=lam0, nus=nu0, mu=mu0, epsilon=1e-8)
    return wrapper, initial

def _cutest_eval_h(wrapper: CutestProblemWrapper, x: np.ndarray) -> np.ndarray:
    prob = wrapper.prob
    if wrapper.meq == 0:
        return np.zeros(0, dtype=float)

    c = np.array(prob.cons(x), dtype=float)
    cl = np.array(prob.cl, dtype=float)

    return c[wrapper.eq_orig_index] - cl[wrapper.eq_orig_index]

def _cutest_eval_g(wrapper: CutestProblemWrapper, x: np.ndarray) -> np.ndarray:
    prob = wrapper.prob
    if wrapper.mineq == 0:
        return np.zeros(0, dtype=float)

    out = np.zeros(wrapper.mineq, dtype=float)
    m = int(getattr(prob, "m", 0))

    c = np.array(prob.cons(x), dtype=float) if m > 0 else np.zeros(0, dtype=float)
    cl = np.array(getattr(prob, "cl", np.zeros(m)), dtype=float) if m > 0 else np.zeros(0)
    cu = np.array(getattr(prob, "cu", np.zeros(m)), dtype=float) if m > 0 else np.zeros(0)

    for k in range(wrapper.mineq):
        if wrapper.trans_is_var_bound[k]:
            j = int(wrapper.trans_var_index[k])
            if bool(wrapper.trans_var_is_upper[k]):
                out[k] = x[j] - float(prob.bu[j])
            else:
                out[k] = float(prob.bl[j]) - x[j]
        else:
            i = int(wrapper.trans_orig_index[k])
            sgn = float(wrapper.trans_sign[k])
            if sgn > 0:
                out[k] = c[i] - float(cu[i])
            else:
                out[k] = float(cl[i]) - c[i]
    return out

def _cutest_eval_jacobians(wrapper: CutestProblemWrapper, x: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    prob = wrapper.prob
    n = int(prob.n)
    m = int(getattr(prob, "m", 0))

    if m > 0:
        _, J = prob.cons(x, gradient=True)
        J = np.array(J, dtype=float)
    else:
        J = np.zeros((0, n), dtype=float)

    if wrapper.meq > 0:
        A = J[wrapper.eq_orig_index, :]
    else:
        A = np.zeros((0, n), dtype=float)

    if wrapper.mineq > 0:
        G = np.zeros((wrapper.mineq, n), dtype=float)
        for k in range(wrapper.mineq):
            if wrapper.trans_is_var_bound[k]:
                j = int(wrapper.trans_var_index[k])
                if bool(wrapper.trans_var_is_upper[k]):
                    G[k, j] = 1.0
                else:
                    G[k, j] = -1.0
            else:
                i = int(wrapper.trans_orig_index[k])
                sgn = float(wrapper.trans_sign[k])
                G[k, :] = sgn * J[i, :]
    else:
        G = np.zeros((0, n), dtype=float)

    return A, G

def _cutest_eval_objgrad(wrapper: CutestProblemWrapper, x: np.ndarray) -> Tuple[float, np.ndarray]:
    f, g = wrapper.prob.obj(x, gradient=True)
    return float(f), np.array(g, dtype=float)

def _cutest_eval_hess_lag(wrapper: CutestProblemWrapper, x: np.ndarray, lambdas: np.ndarray, nus: np.ndarray) -> np.ndarray:
    prob = wrapper.prob
    m = int(getattr(prob, "m", 0))
    if m == 0:
        H = np.array(prob.hess(x), dtype=float)
        return H

    v = np.zeros(m, dtype=float)

    for idx, i in enumerate(wrapper.eq_orig_index):
        v[int(i)] += float(lambdas[idx])

    if wrapper.mineq > 0:
        for k in range(wrapper.mineq):
            if wrapper.trans_is_var_bound[k]:
                continue
            i = int(wrapper.trans_orig_index[k])
            sgn = float(wrapper.trans_sign[k])
            v[i] += sgn * float(nus[k])

    H = np.array(prob.hess(x, v=v), dtype=float)
    return H

def solve_interior_point_cutest(
    wrapper: CutestProblemWrapper,
    initial: InitialPoint,
    config: OptimizationConfig,
    logger: logging.Logger
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, float]:
    n = int(wrapper.prob.n)
    meq = int(wrapper.meq)
    mineq = int(wrapper.mineq)

    x = np.array(initial.x, dtype=float)
    lam = np.array(initial.lambdas, dtype=float)
    nu = np.array(initial.nus, dtype=float)
    s = np.array(initial.s, dtype=float)

    mu = float(initial.mu)

    outer_iter = 0
    while mu > config.mu_min:
        outer_iter += 1
        logger.info("=" * 80)
        logger.info(f"[CUTEst] Outer iteration {outer_iter}: solving barrier problem with μ = {mu:.3e}")
        logger.info("=" * 80)

        for it in range(config.max_newton_iterations):
            f, gradf = _cutest_eval_objgrad(wrapper, x)
            h = _cutest_eval_h(wrapper, x)
            g = _cutest_eval_g(wrapper, x)
            A, G = _cutest_eval_jacobians(wrapper, x)
            H = _cutest_eval_hess_lag(wrapper, x, lam, nu)

            r_dual = gradf + A.T @ lam + G.T @ nu
            r_peq = h
            r_pineq = g + s
            r_cent = nu - (mu / s)

            F = np.concatenate([r_dual, r_peq, r_pineq, r_cent])
            res = float(np.linalg.norm(F))
            logger.info(f"Residual: {res:.6e}")

            if res < config.residual_tolerance_factor * mu:
                logger.info(
                    f"Inner loop converged: ||F|| = {res:.3e} < "
                    f"{config.residual_tolerance_factor} * μ = {(config.residual_tolerance_factor * mu):.3e}\n"
                )
                break

            Z_meq = np.zeros((meq, meq), dtype=float)
            Z_meq_mineq = np.zeros((meq, mineq), dtype=float)
            Z_mineq_meq = np.zeros((mineq, meq), dtype=float)
            Z_mineq = np.zeros((mineq, mineq), dtype=float)
            I_mineq = np.eye(mineq, dtype=float)

            J11 = H
            J12 = A.T
            J13 = G.T
            J14 = np.zeros((n, mineq), dtype=float)

            J21 = A
            J22 = Z_meq
            J23 = Z_meq_mineq
            J24 = np.zeros((meq, mineq), dtype=float)

            J31 = G
            J32 = Z_mineq_meq
            J33 = Z_mineq
            J34 = I_mineq

            J41 = np.zeros((mineq, n), dtype=float)
            J42 = np.zeros((mineq, meq), dtype=float)
            J43 = I_mineq
            J44 = np.diag(mu / (s ** 2))

            top = np.concatenate([J11, J12, J13, J14], axis=1)
            mid1 = np.concatenate([J21, J22, J23, J24], axis=1)
            mid2 = np.concatenate([J31, J32, J33, J34], axis=1)
            bot = np.concatenate([J41, J42, J43, J44], axis=1)
            J = np.concatenate([top, mid1, mid2, bot], axis=0)

            try:
                delta = np.linalg.solve(J, -F)
            except np.linalg.LinAlgError as e:
                logger.warning(f"KKT solve failed: {e}. Stopping inner iterations.")
                break

            dx = delta[:n]
            dlam = delta[n:n+meq]
            dnu = delta[n+meq:n+meq+mineq]
            ds = delta[n+meq+mineq:]

            step = 1.0
            for attempt in range(config.max_step_reductions):
                s_new = s + step * ds
                if np.all(s_new > config.s_margin):
                    break
                step *= config.step_reduction_factor
                if step < config.min_step_size:
                    break

            if step < config.min_step_size:
                logger.warning("Step became too small. Stopping inner iterations.")
                break

            x = x + step * dx
            lam = lam + step * dlam
            nu = nu + step * dnu
            s = s + step * ds

        mu *= config.mu_factor

    logger.info("[CUTEst] Barrier loop finished: μ <= μ_min")
    return x, lam, nu, s, mu

def read_problem_from_sif(file_path: Path) -> Tuple[CutestProblemWrapper, InitialPoint]:
    try:
        import pycutest
    except Exception as e:
        raise ImportError(
            "pycutest is not installed or not importable. "
            "Install CUTEst/SIFDecode/MASTSIF and then `pip install pycutest`."
        ) from e

    sif_path = Path(file_path).expanduser().resolve()
    if not sif_path.exists():
        raise FileNotFoundError(f"SIF file not found: {sif_path}")

    mastsif_dir = Path("./mastsif_local").resolve()
    mastsif_dir.mkdir(parents=True, exist_ok=True)

    problem_name = sif_path.stem.upper()
    dest = mastsif_dir / f"{problem_name}.SIF"
    if dest != sif_path:
        import shutil
        shutil.copy2(sif_path, dest)

    import os
    os.environ["MASTSIF"] = str(mastsif_dir)

    prob = pycutest.import_problem(problem_name, quiet=True)
    wrapper, initial = _build_cutest_wrapper(prob)
    return wrapper, initial


In [ ]:
def main():
    config = OptimizationConfig(
        max_newton_iterations=1000,
        mu_factor=0.1,
        mu_min=1e-6,
        residual_tolerance_factor=0.1,
        s_margin=1e-6,
        max_step_reductions=30,
        step_reduction_factor=0.5,
        min_step_size=1e-10,
        input_file_path="/content/BARD.SIF",
        log_to_file=True
    )

    logger = setup_logger(config.log_to_file)

    if not config.input_file_path:
        raise NotImplementedError("Interactive input not implemented in refactored version")

    input_path = Path(config.input_file_path)

    if input_path.suffix.lower() == ".sif":
        wrapper, initial = read_problem_from_sif(input_path)
        log_initial_parameters(initial, OptimizationProblem(
            target_function=lambda *args: float(wrapper.prob.obj(np.array(args), gradient=False)),
            target_expression=sp.Integer(0),
            x_symbols=[],
            equalities=[],
            inequalities=[],
            lambda_symbols=[],
            nu_symbols=[],
            s_symbols=[]
        ), logger)

        x_opt, lambda_opt, nu_opt, s_opt, mu_final = solve_interior_point_cutest(
            wrapper=wrapper,
            initial=initial,
            config=config,
            logger=logger
        )

        def f_cb(*xvals):
            return float(wrapper.prob.obj(np.array(xvals, dtype=float), gradient=False))

        log_solution(x_opt, lambda_opt, nu_opt, s_opt, mu_final, f_cb, logger)

    else:
        problem, initial = read_problem_from_file(input_path)
        log_initial_parameters(initial, problem, logger)

        x_opt, lambda_opt, nu_opt, s_opt, mu_final = solve_interior_point(
            problem=problem,
            initial=initial,
            config=config,
            logger=logger
        )

        log_solution(x_opt, lambda_opt, nu_opt, s_opt, mu_final, problem.target_function, logger)

if __name__ == "__main__":
    main()


Initial Parameters:

Initial approximation x:
x1 = 1.0
x2 = 1.0
x3 = 1.0

Initial slack variables s:

Initial equality multipliers λ:

Initial μ = 0.1
Initial ε = 1e-08

[CUTEst] Outer iteration 1: solving barrier problem with μ = 1.000e-01
Residual: 8.463082e+01
Residual: 1.682970e+01
Residual: 6.448781e+00
Residual: 2.045809e+00
Residual: 4.408560e-01
Residual: 2.806488e-02
Residual: 7.751495e-04
Inner loop converged: ||F|| = 7.751e-04 < 0.1 * μ = 1.000e-02

[CUTEst] Outer iteration 2: solving barrier problem with μ = 1.000e-02
Residual: 7.751495e-04
Inner loop converged: ||F|| = 7.751e-04 < 0.1 * μ = 1.000e-03

[CUTEst] Outer iteration 3: solving barrier problem with μ = 1.000e-03
Residual: 7.751495e-04
Residual: 1.024006e-06
Inner loop converged: ||F|| = 1.024e-06 < 0.1 * μ = 1.000e-04

[CUTEst] Outer iteration 4: solving barrier problem with μ = 1.000e-04
Residual: 1.024006e-06
Inner loop converged: ||F|| = 1.024e-06 < 0.1 * μ = 1.000e-05

[CUTEst] Outer iteration 5: solving barri